In [2]:
import pandas as pd
import numpy as np 
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

boston = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data", sep="\s+",
                     names=["CRIM","ZN","INDUS","CHAS","NOX","RM","AGE","DIS","RAD","TAX","PRATIO","B","LSTAT","MEDV"])
boston.head()

<>:9: SyntaxWarning: invalid escape sequence '\s'
<>:9: SyntaxWarning: invalid escape sequence '\s'
/var/folders/wb/kbfflkgd24l9kfh0dvs2fsj40000gn/T/ipykernel_64385/3456237854.py:9: SyntaxWarning: invalid escape sequence '\s'
  boston = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data", sep="\s+",


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [3]:
X = boston.drop('MEDV',axis=1).values
Y = boston['MEDV'].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

In [4]:
#creo le features polinomiali per creare il problema dell'overfitting
poly_feats = PolynomialFeatures(degree=2)                               #proprietà polinomiali fino al 2 grado
X_train_poly = poly_feats.fit_transform(X_train)
X_test_poly = poly_feats.transform(X_test)

X_train_poly.shape                                                  #modello estremamente complesso (354 esempi, 105 proprietà)

(354, 105)

In [6]:
#standardizzo il dataset prima della regressione
ss = StandardScaler()
X_train_poly = ss.fit_transform(X_train_poly)
X_test_poly = ss.transform(X_test_poly)

In [9]:
lr = LinearRegression()
lr.fit(X_train_poly, Y_train)
#eseguo la predizione sul training set piuttosto che sul test set
Y_pred_train = lr.predict(X_train_poly)

mse_train = mean_squared_error(Y_pred_train, Y_train)
r2_train = r2_score(Y_pred_train, Y_train)
print("MSE per training: " +str(mse_train))
print("R2 per training: " +str(r2_train))               #Errore molto basso e punteggio molto alto -> modello ottimo? vediamo come performa sul test set

Y_pred_test = lr.predict(X_test_poly)

mse_test = mean_squared_error(Y_pred_test, Y_test)
r2_test = r2_score(Y_pred_test, Y_test)
print("MSE per test: " +str(mse_test))
print("R2 per test: " +str(r2_test))                #Errore molto alto e punteggio basso -> OVERFITTING

MSE per training: 4.116985074931483
R2 per training: 0.9489872442434962
MSE per training: 29.550444547161902
R2 per training: 0.6975947899468182


Regolarizzazione L2 per l'overfitting

In [14]:
from sklearn.linear_model import Ridge
alphas = [0.0001, 0.001, 0.01, 0.1, 1., 10,]            #possibili valori dell'iperparametro alpha
for alpha in alphas:
    print("ALPHA=" +str(alpha))
    model = Ridge(alpha=alpha)
    model.fit(X_train_poly, Y_train)
    
    #eseguo le predizioni sia sul train che sul test set
    Y_pred_train = model.predict(X_train_poly)
    Y_pred_test = model.predict(X_test_poly)
    mse_train = mean_squared_error(Y_pred_train, Y_train)
    mse_test = mean_squared_error(Y_pred_test, Y_test)
    r2_train = r2_score(Y_pred_train, Y_train)
    r2_test = r2_score(Y_pred_test, Y_test)
    print("MSE per training: " +str(mse_train) + " R2 per training: " +str(r2_train))
    print("MSE per test: " +str(mse_test)+ " R2 per test: " +str(r2_test))           #il punteggio migliore si ha per valori di aplha=10
#MSE di test è il più basso (17.15) → il modello ha la migliore capacità di generalizzazione.
#R² di test (0.7458) è alto → il modello spiega bene la varianza senza eccessivo overfitting.
#MSE di training aumenta (8.81) → significa che il modello è più regolarizzato e meno incline al sovradattamento.

ALPHA=0.0001
MSE per training: 4.099263404864359 R2 per training: 0.9491693108936194
MSE per test: 28.9176184637223 R2 per test: 0.7004105533610578
ALPHA=0.001
MSE per training: 4.1135025099425695 R2 per training: 0.9489389789826518
MSE per test: 28.420009267640413 R2 per test: 0.7028044782421446
ALPHA=0.01
MSE per training: 4.2082061272373465 R2 per training: 0.9474714022331885
MSE per test: 26.81329501829148 R2 per test: 0.7093928621764054
ALPHA=0.1
MSE per training: 4.747028830953304 R2 per training: 0.9399140725675383
MSE per test: 23.63175511737035 R2 per test: 0.7268152606270204
ALPHA=1.0
MSE per training: 5.87594730534178 R2 per training: 0.9231147673228387
MSE per test: 17.634584627531336 R2 per test: 0.77084019159856
ALPHA=10
MSE per training: 8.812755521737886 R2 per training: 0.8752076591659455
MSE per test: 17.159715774774316 R2 per test: 0.7458268471061418


Regolarizzazione L2 per l'overfitting

In [13]:
#importo il dataset lasso
from sklearn.linear_model import Lasso
#riutilizzo il codice del blocco sopra
alphas = [0.0001, 0.001, 0.01, 0.1, 1., 10,]            #possibili valori dell'iperparametro alpha
for alpha in alphas:
    print("ALPHA=" +str(alpha))
    model = Lasso(alpha=alpha)
    model.fit(X_train_poly, Y_train)
    
    #eseguo le predizioni sia sul train che sul test set
    Y_pred_train = model.predict(X_train_poly)
    Y_pred_test = model.predict(X_test_poly)
    mse_train = mean_squared_error(Y_pred_train, Y_train)
    mse_test = mean_squared_error(Y_pred_test, Y_test)
    r2_train = r2_score(Y_pred_train, Y_train)
    r2_test = r2_score(Y_pred_test, Y_test)
    print("MSE per training: " +str(mse_train) + " R2 per training: " +str(r2_train))
    print("MSE per test: " +str(mse_test)+ " R2 per test: " +str(r2_test))       #punteggio migliore per aplha=0.1
#MSE di test è minimo (19.29) -> migliore generalizzazione.
#R2 di test (0.6899) è alto -> indica che il modello spiega bene la varianza dei dati di test.
#MSE di training aumenta un po', il modello è più semplice e meno incline all'overfitting.

ALPHA=0.0001
MSE per training: 5.391123652697103 R2 per training: 0.9319016473138058
MSE per test: 29.701776720600684 R2 per test: 0.6578075263506309
ALPHA=0.001
MSE per training: 5.407317548867132 R2 per training: 0.931445534004468
MSE per test: 28.78801855730682 R2 per test: 0.6648236745082188
ALPHA=0.01
MSE per training: 6.063858816900331 R2 per training: 0.9208898979105544
MSE per test: 22.93324201265608 R2 per test: 0.7193170700220337
ALPHA=0.1
MSE per training: 11.833211121207539 R2 per training: 0.8249529309910117
MSE per test: 19.296152342816367 R2 per test: 0.6899597066684204
ALPHA=1.0
MSE per training: 21.590985067091985 R2 per training: 0.5286025713768555
MSE per test: 27.258043145129136 R2 per test: 0.37369795557404095
ALPHA=10
MSE per training: 84.76451346994796 R2 per training: 0.0
MSE per test: 83.76673764512785 R2 per test: 0.0


/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.490e+02, tolerance: 3.001e+00
  model = cd_fast.enet_coordinate_descent(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.866e+02, tolerance: 3.001e+00
  model = cd_fast.enet_coordinate_descent(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.049e+02, toler

Elasticnet per combinare le regolarizzazioni L1 ed L2

In [15]:
from sklearn.linear_model import ElasticNet
#riutilizzo lo stesso codice
alphas = [0.0001, 0.001, 0.01, 0.1, 1., 10,]            #possibili valori dell'iperparametro alpha
for alpha in alphas:
    print("ALPHA=" +str(alpha))
    model = ElasticNet(alpha=alpha, l1_ratio=0.5)       #l1ratio per dare lo stesso peso alle due regolarizzazioni
    model.fit(X_train_poly, Y_train)
    
    #eseguo le predizioni sia sul train che sul test set
    Y_pred_train = model.predict(X_train_poly)
    Y_pred_test = model.predict(X_test_poly)
    mse_train = mean_squared_error(Y_pred_train, Y_train)
    mse_test = mean_squared_error(Y_pred_test, Y_test)
    r2_train = r2_score(Y_pred_train, Y_train)
    r2_test = r2_score(Y_pred_test, Y_test)
    print("MSE per training: " +str(mse_train) + " R2 per training: " +str(r2_train))
    print("MSE per test: " +str(mse_test)+ " R2 per test: " +str(r2_test)) 
#il modello performa meglio rispetto alle normaizzazioni l1 e 2, miglior punteggio per aplha=0.1
#E FONDAMENTALE AVERE IL DATASET SULLA STESSA SCALA, QUINDI NORMALIZZAZIONE O STANDARDIZZAZIONE COME QUI

ALPHA=0.0001
MSE per training: 5.391059281137915 R2 per training: 0.9318342684736182
MSE per test: 29.466017582883342 R2 per test: 0.659769605157168
ALPHA=0.001
MSE per training: 5.463124643400424 R2 per training: 0.9301999073618576
MSE per test: 26.23899793865848 R2 per test: 0.6869076041372015
ALPHA=0.01
MSE per training: 6.669947875220289 R2 per training: 0.9109107527583874
MSE per test: 15.78442472698635 R2 per test: 0.784377065673517
ALPHA=0.1
MSE per training: 12.092531251957974 R2 per training: 0.8163279674919248
MSE per test: 20.123693597792258 R2 per test: 0.6717322148874648
ALPHA=1.0
MSE per training: 21.178857007859765 R2 per training: 0.5493858787176467
MSE per test: 27.923580301576497 R2 per test: 0.37588929686229
ALPHA=10
MSE per training: 70.28359861834348 R2 per training: -64.51072945797367
MSE per test: 69.68198552608109 R2 per test: -61.422059153377


/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.665e+02, tolerance: 3.001e+00
  model = cd_fast.enet_coordinate_descent(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.014e+03, tolerance: 3.001e+00
  model = cd_fast.enet_coordinate_descent(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.930e+02, toler